In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
Crime_5_7 = pd.read_csv('C:\\Users\\Ajit\\Documents\\Chicago_Crimes_2005_to_2007.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0) 
Crime_8_11 = pd.read_csv('C:\\Users\\Ajit\\Documents\\Chicago_Crimes_2008_to_2011.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0) 
Crime_12_17 = pd.read_csv('C:\\Users\\Ajit\\Documents\\Chicago_Crimes_2012_to_2017.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0)

In [3]:
ZipCode_Data = pd.read_csv('C:\\Users\\Ajit\\Documents\\Chicago_Zip_Data.csv.', header = 0)

In [4]:
Crime_Data = [Crime_5_7, Crime_8_11, Crime_12_17]
del Crime_5_7
del Crime_8_11
del Crime_12_17
Crime_Data = pd.concat(Crime_Data,axis = 0)

In [5]:
Crime_Data.drop_duplicates(subset=['ID', 'Case Number'], inplace=True)

In [6]:
Crime_Data.drop(['Unnamed: 0','Case Number','IUCR','FBI Code','Location',
                 'X Coordinate','Y Coordinate','Location'], inplace = True, axis = 1)

In [7]:
Crime_Data.Date = pd.to_datetime(Crime_Data.Date, format = '%m/%d/%Y %I:%M:%S %p')
Crime_Data['Updated On'] = pd.to_datetime(Crime_Data['Updated On'], format = '%m/%d/%Y %I:%M:%S %p')
Crime_Data.index = pd.DatetimeIndex(Crime_Data.Date)

In [8]:
#will look into time it takes for an arrest later, so creating column for it
#turning other parts of data into categorical parts for later machine learning applications
Crime_Data['ArrestTime'] = Crime_Data['Updated On'] - Crime_Data.Date
Crime_Data['Primary Type'] = pd.Categorical(Crime_Data['Primary Type'])
Crime_Data['Description'] = pd.Categorical(Crime_Data['Description'])
Crime_Data['Location Description'] = pd.Categorical(Crime_Data['Location Description'])

In [9]:
Groups = Crime_Data.groupby(Crime_Data['Primary Type'])
Groups = dict(list(Groups))

In [10]:
#grouping sex crimes together to look into
SexCrimes_Data = [Groups['CRIM SEXUAL ASSAULT'], Groups['HUMAN TRAFFICKING'],
                  Groups['OFFENSE INVOLVING CHILDREN'], Groups['PROSTITUTION'],
                  Groups['SEX OFFENSE'], Groups['STALKING']]
SexCrimes_Data = pd.concat(SexCrimes_Data, axis = 0)
del Groups
del Crime_Data

In [11]:
SexCrimeZipData = pd.read_csv('C:\\Users\\Ajit\\Documents\\ChicagoSexCrimesZipCodeData.csv',header = 0)
SexCrimes_Data = SexCrimes_Data.dropna(axis = 0, how = 'any')
SexCrimeZipData.index = pd.DatetimeIndex(SexCrimes_Data.index)

In [12]:
#adding zipcode data
SexCrimes_Data['ZipCode'] = SexCrimeZipData['ZipCode']

In [13]:
#converting data for each zipcode into list zipdata
#recently learned that python is both amazing for memory saving and also really annoying for coding...
G = dict(ZipCode_Data)
keys = list(G.keys())
listdata = []
zipdata = []
for i in range(len(G[keys[0]])):
    for j in range(1,len(keys)):
        listdata.append(G[keys[j]][i])
    zipdata.append(listdata)
    listdata = []

In [14]:
#binary search algorithm to find zip code
def BinarySearch(item, L, data, low, high):
    mid = int((low + high) / 2)
    if (item == L[mid]):
        return data[mid]
    elif (low > high):
        return None
    else:
        if (item < L[mid]):
            return BinarySearch(item, L, data, low, mid - 1)
        else:
            return BinarySearch(item, L, data, mid + 1, high)

In [15]:
SC_Data_Zip = list(SexCrimes_Data.ZipCode)
zdata = [None] * (SexCrimeZipData.ZipCode.shape[0])
for i in range(SexCrimeZipData.ZipCode.shape[0]):
    z = BinarySearch(SC_Data_Zip[i], ZipCode_Data.ZipCode, zipdata, 0, 84)
    zdata[i] = z

In [16]:
#creating a data frame, and concatenating the two dataframes
zdata = pd.DataFrame(zdata[:], columns=keys[1:])
zdata.index = SexCrimes_Data.index
Data = [SexCrimes_Data, zdata]
Data = pd.concat(Data, axis = 1)

In [17]:
#I am going to see if I can isolate any pattern about where the crimes are centered around.
#I will start by using an unsupervised clustering algorithm to see if there are areas of high concentrations of sex crimes.
#I can look at it cumulatively first, then segment by year and see if the clusters are moving/changing in anyway.
#I can then see if there is any overlap between the different sex crime types that I have selected, 
#and if there is, it might mean that the centroids of these clusters are hits for where the crime originates from.
#In the previous part, I tried to see if there was any way to discern whether an arrest would be made or not;
#therefore, in this one I will also try to do that given the new information about the zip code data.
#One of the things I will be looking for is if there is any relationship between sex trafficking and the other crimes
#after a few years have passed.

#No attempt was made to determine the exact location of the crimes because it was stated by the CPD in the dataset not to

In [18]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

In [19]:
CH_indices = []
CH_indices = np.array(CH_indices)
Data.drop(['ID','Block','Date','Description','Location Description','Primary Type','Updated On','ArrestTime'], 
          inplace = True, axis = 1)

In [23]:
for i in range(2,25):
    KMod = KMeans(n_clusters =i, random_state = 1, n_jobs = -1).fit(Data)
    labels = KMod.labels_
    CH_indices = np.append(CH_indices, metrics.silhouette_score(Data, labels, metric = 'cityblock'))

MemoryError: 

2